# Basic information

## three fanspages
* imeifoods id='202586376996' 
* iLoveKCmilk id='180080052032125' 
* Dominos.tw id='115361891807579'

In [1]:
import pyes
import json
from IPython.core.display import display, HTML


host = '192.168.1.100:9200'
conn = pyes.es.ES(host)

fnames = ['imeifoods', 'iLoveKCmilk', 'Dominos.tw']
fids = ['202586376996', '180080052032125', '115361891807579']

tqs = [pyes.query.TermQuery(field="fid", value=fid) for fid in fids]
rs = [conn.search(query=tq, indices='facebook_nested', doc_types='fanpage') for tq in tqs]

print 'I choose three fanpages to analyze as following'
for r, name in zip(rs, fnames):
    display(HTML('<a href='+ json.dumps(r[0]['link'], indent=2) + ' target="_blank">' + name + '</a>'))

I choose three fanpages to analyze as following


## searching range (2015 whole year)

In [2]:
ESR = pyes.ESRange(field='created_time', from_value='2015-01-02T00:00:00+0800', to_value='2015-12-31T23:59:59+0800', 
                   include_lower=True, include_upper=False) # use whole data in 2015
rq = pyes.query.RangeQuery(qrange=ESR)
bqs = [pyes.query.BoolQuery().add_must(rq).add_must(tq) for tq in tqs]

## # of comments

In [3]:
res = [conn.search(query=bq, indices='facebook_nested', doc_types='comment') for bq in bqs]
print [len(r) for r in res]

[25541, 15183, 35130]


## # of posts

In [4]:
res = [conn.search(query=bq, indices='facebook_nested', doc_types='post') for bq in bqs]
print [len(r) for r in res]

[207, 104, 364]


## aggregate by 12 months

In [5]:
DHAgg = pyes.aggs.DateHistogramAgg('month', field='created_time', interval='month')

qs = [pyes.query.Search(bq) for bq in bqs]
for q in qs:
    q.agg.add(DHAgg)

## # of comments in 12 months

In [6]:
rs = [conn.search(query=q, indices='facebook_nested', doc_types='comment') for q in qs]
comment_month = [[rs[i].aggs['month']['buckets'][m]['doc_count'] for m in range(12)] for i in range(3)]
print comment_month

[[654, 903, 5042, 1150, 1461, 1821, 5581, 480, 1149, 2685, 2560, 2055], [5747, 1237, 2, 609, 2385, 2524, 339, 169, 464, 1535, 22, 150], [1296, 679, 1797, 964, 2762, 1655, 2388, 4970, 4386, 3053, 5659, 5521]]


## # of posts in 12 months

In [7]:
rs = [conn.search(query=q, indices='facebook_nested', doc_types='post') for q in qs]
post_month = [[rs[i].aggs['month']['buckets'][m]['doc_count'] for m in range(12)] for i in range(3)]
print post_month

[[16, 15, 15, 12, 14, 15, 28, 14, 17, 22, 15, 24], [17, 8, 0, 9, 10, 13, 11, 10, 8, 12, 1, 5], [40, 25, 26, 38, 32, 33, 35, 29, 25, 26, 30, 25]]
